In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
from torchvision.models import alexnet, AlexNet_Weights
from torchvision import datasets
from pathlib import Path
import matplotlib.pyplot as plt
plt.rcParams["savefig.bbox"] = 'tight'
from torchvision.transforms import v2
from torchvision.io import read_image
torch.manual_seed(1) # seed
from helpers import plot, random_combination
import os
import shutil
import time
from tempfile import TemporaryDirectory
from PIL import Image

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
weights = AlexNet_Weights.DEFAULT
model = alexnet(weights = None) # weights = weights
standard_preprocess = weights.transforms()

In [ ]:
data_transforms = {
                    'train': v2.Compose([
                            v2.RandomApply([v2.RandAugment()], p = .5),
                            v2.RandomResizedCrop(size=(224, 224), antialias=True),
                            v2.RandomHorizontalFlip(p=0.5),
                            v2.ToTensor(),
                            v2.ToDtype(torch.float32, scale=True),
                            v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]),
                   'val': standard_preprocess}
data_dir = './Dataset_Categorised/'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

def visualize_model_predictions(model,img_path):
    was_training = model.training
    model.eval()

    img = Image.open(img_path)
    img = standard_preprocess(img)
    img = img.unsqueeze(0)
    img = img.to(device)

    with torch.no_grad():
        outputs = model(img)
        _, preds = torch.max(outputs, 1)

        ax = plt.subplot(2,2,1)
        ax.axis('off')
        ax.set_title(f'Predicted: {class_names[preds[0]]}')
        imshow(img.cpu().data[0])

        model.train(mode=was_training)

def predict(model,img_path):
    was_training = model.training
    model.eval()

    img = Image.open(img_path)
    img = data_transforms['val'](img)
    img = img.unsqueeze(0)
    img = img.to(device)

    with torch.no_grad():
        outputs = model(img)
        _, preds = torch.max(outputs, 1)

        return preds

In [ ]:
df_train_labels = pd.read_csv('./Dataset/train_labels.csv')

Initialise an AlexNet (with weights) and see if we can train anything.

In [ ]:
model_classifier = nn.Sequential(model, nn.Linear(list(model.children())[-1][-1].out_features, 4))

In [ ]:
# Push to GPU
model_classifier.to(device)

Build Transforms

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=5):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
        
    return model

In [ ]:
optimizer_ft = optim.Adam(model_classifier.parameters(), lr=1e-4)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.8)
criterion = nn.CrossEntropyLoss()

In [ ]:
model_ft = train_model(model_classifier, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=60)

In [ ]:
visualize_model(model_ft)

plt.ioff()
plt.show()

In [ ]:
visualize_model_predictions(
    model_ft,
    img_path='./Dataset/test/aapvlatnvr.jpeg'
)

plt.ioff()
plt.show()

In [ ]:
predict(model_ft, './Dataset/test/aapvlatnvr.jpeg').cpu().item()

In [ ]:
all_test_files_jpeg = listdir('./Dataset/test/')
all_test_files = ['./Dataset/test/' + i for i in all_test_files_jpeg]

In [ ]:
all_test_pred = []
for file_str in all_test_files:
    all_test_pred.append(predict(model_ft, file_str).cpu().item())

In [ ]:
pd.DataFrame({'filename': all_test_files_jpeg, 'category':all_test_pred }).to_csv('submission_1_basic_alexnet_augmented.csv', index=False)

In [ ]:
len(all_test_pred)